# Iris Dataset Testing

In [2]:
import sys  
sys.path.insert(0, '../')
import deep_forest
import preprocess
import torch as th
from torch import nn as nn
import matplotlib.pyplot as plt
%matplotlib inline
from math import pi
import seaborn as sns
from tqdm import tqdm
sns.set_theme("notebook")
sns.set_style('whitegrid')

## Generate Data
The data is the UCl iris dataset. Here, 100% of the data is used for training.

In [3]:
train_data, train_labels, test_data, test_labels = preprocess.get_data(100)

## Deep Forest
Here, the model is instantiated (using cuda or the CPU) and trained.

In [4]:
model = deep_forest.DeepForest(100, 2, 4, 0.5, 10)
device = th.device("cuda" if th.cuda.is_available() else "cpu")
model = model.to(device)
train_data = train_data.to(device)
train_labels = train_labels.to(device)

In [ ]:
optimizer = th.optim.Adam(model.parameters())
pbar = tqdm(range(2500))
for i in pbar:
    model.populate_best(train_data[:, :], train_labels[:])
    optimizer.zero_grad()
    
    loss = model.loss(train_data[:, :], train_labels[:], device)
    loss.backward()
    optimizer.step()

    
    pbar.set_description("EPOCH %d || Acc: %s || Loss: %s" % (i, str(th.mean((model.forward(train_data[:, :], device) == train_labels[:]).float())), str(loss)))

print("\n==============\nFINAL ACC: %s" % str(th.mean((model.forward(train_data[:, :], device) == train_labels[:]).float())))

  0%|          | 0/2500 [00:00<?, ?it/s]/Users/kyranflynn/miniconda3/lib/python3.8/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
EPOCH 29 || Acc: tensor(0.6667) || Loss: tensor([672.3883], grad_fn=<AddBackward0>):   1%|          | 30/2500 [00:16<23:50,  1.73it/s]

# MLP Baseline
Here, a baseline feedforward is used on thr Iris dataset for comparison.

In [ ]:
mlp = nn.Sequential(
    nn.Linear(4, 15),
    nn.LeakyReLU(),
    nn.Linear(15, 15),
    nn.LeakyReLU(),
    nn.Linear(15, 3),
    nn.Softmax()
)

optimizer = th.optim.Adam(mlp.parameters())
for i in range(1000):
    optimizer.zero_grad()

    preds = mlp(train_data[:, :])
    loss = nn.functional.cross_entropy(preds, (train_labels[:].type(th.LongTensor)).to(device))
    loss.backward()
    optimizer.step()

    pbar.set_description("EPOCH %d || Acc: %s || Loss: %s" % (i, str(th.mean((model.forward(train_data[:, :], device) == train_labels[:]).float())), str(loss)))
        
print("\n==============\nFINAL ACC: %s" % str(th.mean((th.argmax(mlp(train_data[:]), 1) == train_labels[:]).float())))


# Standard Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2)
clf.fit(train_data[:, :].numpy(), train_labels[:].numpy())
print("Accuracy: " + clf.score(train_data[:, :].numpy(), train_labels[:].numpy()))